# Python: Identify areas of potential foliage influence on railway network

In [ ]:
import geopandas as gpd
import shapely
import folium

## Add [WFS](https://python-gis-book.readthedocs.io/en/latest/part2/chapter-09/nb/01-retrieving-data-from-wfs.html) layer in GeoPandas and clip it to area of interest

In [ ]:
railway_wfs_url = 'https://geoserver.geonet-mrn.de/xdatatogo/db_strecken/ows?'\
                    'service=WFS&'\
                    'version=2.0.0&'\
                    'request=GetFeature&'\
                    'typeNames=xdatatogo:db_strecken&'\
                    'outputFormat=json'
railway_network = gpd.read_file(railway_wfs_url)

Visualise downloaded whole data in map

In [ ]:
map = railway_network.explore()
map

Clip the railway network to WGS 84 ([EPSG:4326](https://epsg.io/4326)) bounding box of Lower Saxony. The BBOX is be defined by its boundaries: `long_min, lat_min, long_max, lat_max`.

In [ ]:
bbox = (6.345854, 51.295232, 11.598078, 54.13791)
railway_network = railway_network.clip(bbox)

In [ ]:
map = railway_network.explore()
#**TODO** Add BBOX polygon
#bbox_polygon = shapely.envelope(shapely.MultiPoint([(bbox[0], bbox[1]), (bbox[2], bbox[3])]))
#folium.GeoJson(bbox_polygon).add_to(map)
map

## Project the extracted layer to UTM32 ([EPSG:25832](https://epsg.io/25832)) in order to be able to use "metre" unit for measuring and buffering

In [ ]:
print("Previous CRS:", railway_network.crs)
railway_network = railway_network.to_crs(epsg=25832)
print("Projected CRS:", railway_network.crs)

## Create a buffer around the railway tracks

In [ ]:
buffer_distance_m = 20
# duplicate layer to separately store it with a buffered geometry
railway_network_buffer = railway_network.copy()
railway_network_buffer["geometry"] = railway_network["geometry"].buffer(buffer_distance_m)

Inspect the data table if it contains `Polygon` geometries now instead of `LineString` or `MultiLineString`.

In [ ]:
railway_network_buffer.head()

## Identifying areas of foliage influence on railway network

Load [forest dataset from WFS](https://mis.bkg.bund.de/trefferanzeige?docuuid=75C069E4-D760-49FF-BD71-5188CF81B4D9) of BKG.

The relevant dataset "AX_Wald" is identified by the cryptic layer name `dlm250:objart_43002_f`.

This governmental server returns geometries referenced by default in EPSG:25832. Thus, we can also skip an additional re-projection.

We skip a subsequent clipping step by directly including the BBOX as spatial filter in the WFS request. Because our BBOX is specified in WGS 84 longitude/latitude, we have to explicitly provide the reference system identifier `EPSG:4326` for the server to correctly interpret the coordinates. 

In [ ]:
bbox_query_parameter = ",".join(str(coord) for coord in bbox) + ",EPSG:4326"
forest_wfs_url = 'https://sgx.geodatenzentrum.de/wfs_dlm250?'\
                    'service=WFS&'\
                    'version=2.0.0&'\
                    'request=GetFeature&'\
                    'typeNames=dlm250:objart_43002_f&'\
                    'outputFormat=json&'\
                    'bbox=' + bbox_query_parameter
forest_patches = gpd.read_file(forest_wfs_url)

Perform spatial [intersection](https://en.wikipedia.org/wiki/Vector_overlay#Operators) of our railway network with the forest patches

In [ ]:
result = gpd.overlay(railway_network_buffer, forest_patches, how='intersection')

## Plot the result

**TODO** Add all layers to map

In [ ]:
map = result['geometry'].explore(name='susceptible railway areas', color='red')
#folium.GeoJson(railway_network).add_to(map)
folium.LayerControl().add_to(map)
map

## Optional: export result to a file

In [ ]:
output_file = 'railway_network_intersect_forest.json'
result.to_file(output_file)
print('File exported successfully.')